In [1]:
import time
import datetime

from textwrap import dedent

from pow.app.client import (
    initiate,
    stop,
    get_status,
    get_generated,
    get_validated,
    validate,
    init_gen,
    init_val,
    start_validation,
)

import hashlib

from pow.compute.stats import estimate_R_from_experiment
from pow.compute.compute import ProofBatch
from pow.models.utils import Params


date_str = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

CHAIN_HASH = hashlib.sha256(date_str.encode()).hexdigest()

PUBLIC_KEY_1 = f"pub_key_1_{date_str}"
PUBLIC_KEY_2 = f"pub_key_2_{date_str}"
PUBLIC_KEY_3 = f"pub_key_3_{date_str}"

BATCH_SIZE_1 = 8000
BATCH_SIZE_2 = 3500
BATCH_SIZE_3 = 15000

# 1 good nonce for 5000 checked
R_TARGET = 1.387185311663014

# For n = 20000
# 1 good nonce for 2000 checked
R_TARGET = 1.3974563001625417

# # 1 good nonce for 500 checked
R_TARGET = 1.3931334028176634


# n = 20000
# 1 good nonce for 100 checked
R_TARGET = 1.4024477475855244


URL_1 = "http://34.170.104.200:8080"
URL_2 = "http://34.45.171.196:8080"
URL_3 = "http://34.69.126.67:8080"


# R_ESTIMATE = estimate_R_from_experiment(
#     n=8192,
#     P=0.0005,
#     num_samples=50000
# )

# print(R_ESTIMATE)


In [104]:

R_ESTIMATE = estimate_R_from_experiment(
    n=8192,
    P=0.001,
    num_samples=50000
)

print(R_ESTIMATE)



1.3900514434415774


In [145]:



# R = 1.3900514434415774
# params = Params(
#     # dim=1024,
#     # vocab_size=4096,
#     # n_layers=16,
#     # n_heads=16,
#     # n_kv_heads=16
# )



BATCH_SIZE_1 = 10000
BATCH_SIZE_3 = 14000
R_TARGET = 1.39005144344157
params = Params(
    dim=512,
    n_layers=64,
    n_heads=64,
    n_kv_heads=64,
    vocab_size=8192,
    ffn_dim_multiplier=32.0,
    multiple_of=1024,
    norm_eps=1e-05, 
    rope_theta=500000.0,
    use_scaled_rope=True,
    seq_len=4
)

In [146]:
# init_gen(URL_1, CHAIN_HASH, PUBLIC_KEY_1, BATCH_SIZE_1, R_TARGET, params)
# init_gen(URL_2, CHAIN_HASH, PUBLIC_KEY_2, BATCH_SIZE_2, R_TARGET, params)
init_gen(URL_3, CHAIN_HASH, PUBLIC_KEY_3, BATCH_SIZE_3, R_TARGET, params)

In [73]:
# init_val(URL_2, CHAIN_HASH, PUBLIC_KEY_2, BATCH_SIZE_2, R_TARGET, params)
init_val(URL_1, CHAIN_HASH, PUBLIC_KEY_1, BATCH_SIZE_1, R_TARGET)
# init_gen(URL_3, CHAIN_HASH, PUBLIC_KEY_3, BATCH_SIZE_3, R_TARGET, params)

[]

In [57]:
URL_l4x2 = "http://34.173.45.153:8080"
BATCH_SIZE_L4X2 = 15000
PUBLIC_KEY_L4X2 = f"pub_key_l4x2_{date_str}"

URL_P4 = "http://35.232.233.188:8080"
BATCH_SIZE_P4 = 8000
PUBLIC_KEY_P4 = f"pub_key_p4_{date_str}"

URL_A100 = "http://34.121.141.55:8080"
BATCH_SIZE_A100 = 28000
PUBLIC_KEY_A100 = f"pub_key_a100_{date_str}"

R_TARGET = 1.39005144344157
params = Params(
    dim=512,
    n_layers=64,
    n_heads=128,
    n_kv_heads=128,
    vocab_size=8192,
    ffn_dim_multiplier=16.0,
    multiple_of=1024,
    norm_eps=1e-05,
    rope_theta=500000.0,
    use_scaled_rope=True,
    seq_len=4
)

try:
    stop(URL_l4x2)
except Exception as e:
    print(e)

try:
    stop(URL_P4)
except Exception as e:
    print(e)

try:
    stop(URL_A100)
except Exception as e:
    print(e)

init_gen(URL_l4x2, CHAIN_HASH, PUBLIC_KEY_L4X2, BATCH_SIZE_L4X2, R_TARGET, params)
init_gen(URL_P4, CHAIN_HASH, PUBLIC_KEY_P4, BATCH_SIZE_P4, R_TARGET, params)
init_gen(URL_A100, CHAIN_HASH, PUBLIC_KEY_A100, BATCH_SIZE_A100, R_TARGET, params)


{'public_key': '', 'chain_hash': '', 'nonces': [], 'dist': []}

In [12]:
from pow.app.client import start_generation

start_generation(URL_l4x2)


{'status': 'GENERATING'}

In [131]:
stop(URL_1)
stop(URL_3)


{'status': 'STOPPED'}

In [ ]:
# params = Params(
#     dim=2048,
#     vocab_size=20000,
#     n_layers=32,
#     n_heads=32,
#     n_kv_heads=32
# )

# # init_gen(URL_1, CHAIN_HASH, PUBLIC_KEY_1, BATCH_SIZE_1, R_TARGET, params)

In [ ]:
to_validate = get_generated(URL_3)
to_validate = ProofBatch(**to_validate)
print(to_validate.nonces)

In [ ]:
for x in to_validate.split(128):
    validate(URL_2, x.__dict__)
    # validate(URL_1, x.__dict__)



In [ ]:
validated_1 = []
validated_2 = []

In [ ]:
validated_1.extend(get_validated(URL_2))
# validated_2.extend(get_validated(URL_2))

print(len(validated_1))
# print(len(validated_2))

In [ ]:
val_1_nonce2dist = {}
for batch in validated_1:
    for nonce, dist in zip(batch["nonces"], batch["dist"]):
        val_1_nonce2dist[nonce] = dist

val_2_nonce2dist = {}
for batch in validated_2:
    for nonce, dist in zip(batch["nonces"], batch["dist"]):
        val_2_nonce2dist[nonce] = dist

In [ ]:
for nonce, dist in zip(to_validate.nonces, to_validate.dist):
    if nonce not in val_1_nonce2dist:
        print(f"Nonce {nonce} not in val_1")
    if nonce not in val_2_nonce2dist:
        print(f"Nonce {nonce} not in val_2")


    val_1_dist = val_1_nonce2dist[nonce]
    # val_2_dist = val_2_nonce2dist[nonce]
    diff = dist - val_1_dist
    print(f"Nonce: {nonce} | dist_gen: {dist:.7f} | dist_val_1: {val_1_dist:.7f} | diff: {diff:.7f} | {val_1_dist < R_TARGET}")


In [20]:
gen = range(1, float('inf'), 2)

TypeError: 'float' object cannot be interpreted as an integer

In [21]:
from itertools import count

In [ ]:
next()